In [ ]:
from portfolio_tester.config import Asset, Portfolio, SamplerConfig, SimConfig, Goal
from portfolio_tester.data.fetchers import fetch_prices_monthly, prep_returns_and_macro, fetch_fred_series
from portfolio_tester.sampling.bootstrap import ReturnSampler
from portfolio_tester.engine.simulator import MonteCarloSimulator
from portfolio_tester.analytics.metrics import cagr, twrr_annualized, max_drawdown, sharpe_sortino
from portfolio_tester.analytics.risk import efficient_frontier, portfolio_annual_stats, single_asset_stats, max_sharpe_portfolio, risk_free_annual
from portfolio_tester.viz.charts import plot_allocation_donut, plot_efficient_frontier, plot_end_balance_hist, plot_percentile_bands, plot_survival_curve
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
# 1) Portfolio (MVP)
p = Portfolio([
    Asset("VTI","Vanguard Total Stock Market ETF",0.30),
    Asset("TLT","iShares 20+ Year Treasury Bond ETF",0.40),
    Asset("IEF","iShares 7-10 Year Treasury Bond ETF",0.15),
    Asset("GSG","iShares S&P GSCI Commodity-Indexed Trust",0.075),
    Asset("GLD","SPDR Gold Shares",0.075),
])


In [ ]:

# 2) Configs
sim_cfg = SimConfig(horizon_months=30*12, n_sims=100, starting_balance=1_000_000)  # start with 100 sims
sam_cfg = SamplerConfig(mode="single_year", block_years=1, seed=42)

goals = [
    # Withdraw $4,000/mo starting in 1 year, for 30 years, inflation-indexed (real)
    Goal("Retirement Withdrawals", amount=-4000, start_month=12, frequency=12, repeats=30*12, real=True),
]


In [ ]:
# 3) Data
tickers = p.tickers()
prices_m = fetch_prices_monthly(tickers)
rets_m, infl_m, rf_m = prep_returns_and_macro(prices_m)



In [ ]:
# 4) Sample paths
sampler = ReturnSampler(rets_m, infl_m)
R_paths, CPI_paths = sampler.sample(sim_cfg.horizon_months, sim_cfg.n_sims, sam_cfg)

In [ ]:
# 5) Run simulation
sim = MonteCarloSimulator(weights=p.weights_vector(), starting_balance=sim_cfg.starting_balance, rebalance_every_months=sim_cfg.rebalance_every_months)
out = sim.run_with_cashflows(R_paths, CPI_paths, goals)

In [ ]:
# 6) Simple summary
surv = (out["failure_month"] == -1).mean()
cagr_vals = cagr(out["balances"], sim_cfg.horizon_months)
twrr_vals = twrr_annualized(out["twrr_monthly"])
mdd_vals = max_drawdown(out["balances"])

def pct(x): return f"{100*x:.1f}%"
print("=== Monte Carlo Summary (100 sims) ===")
print(f"Survival rate: {pct(surv)}")
print(f"End balance (nominal) median: ${np.median(out['balances'][:,-1]):,.0f}")
print(f"CAGR median: {np.nanmedian(cagr_vals):.2%}")
print(f"TWRR median: {np.nanmedian(twrr_vals):.2%}")
print(f"Max Drawdown median: {np.median(mdd_vals):.1%}")
print("Percentiles (10/50/90) - End Balance:",
        [f"${v:,.0f}" for v in np.percentile(out['balances'][:,-1], [10,50,90])])

In [ ]:
prices_m.head().to_csv("prices_m_head.csv", index=True)


In [ ]:
prices_m = fetch_prices_monthly(tickers)
cpi = fetch_fred_series("CPIAUCSL", start=rets_m.index.min(), end=rets_m.index.max())
tb3 = fetch_fred_series("TB3MS", start=rets_m.index.min(), end=rets_m.index.max())


In [ ]:
# quick alignment check
rets, infl, rf = prep_returns_and_macro(prices_m)
print("indexes equal:", rets.index.equals(infl.index) and rets.index.equals(rf.index))
print("rets index min/max:", rets.index.min(), rets.index.max())
print("infl missing:", infl.isna().any(), "rf missing:", rf.isna().any())
# show any index differences
print("extra in infl:", infl.index.difference(rets.index))
print("extra in rf:", rf.index.difference(rets.index))

In [ ]:
import pandas as pd
from collections import defaultdict

def check_full_year_coverage(rets, infl, rf):
    """Return True if each input has all 12 months for every observed year."""
    datasets = {"rets": rets, "infl": infl, "rf": rf}

    def missing_months(idx):
        idx = pd.DatetimeIndex(idx)
        by_year = defaultdict(set)
        for ts in idx:
            by_year[ts.year].add(ts.month)
        return {
            year: sorted(set(range(1, 13)) - months)
            for year, months in by_year.items()
            if len(months) != 12
        }

    coverage = {name: missing_months(obj.index) for name, obj in datasets.items()}
    all_full = all(len(missing) == 0 for missing in coverage.values())
    if all_full:
        print("All series have complete 12-month coverage for every year.")
    else:
        print("Missing months detected:")
        for name, missing in coverage.items():
            if not missing:
                continue
            print(f"  {name}:")
            for year, months in sorted(missing.items()):
                month_list = ', '.join(f"{m:02d}" for m in months)
                print(f"    {year}: {month_list}")
    return all_full, coverage


In [ ]:
testa=check_full_year_coverage(rets, infl, rf)
testa

In [ ]:
R_paths.shape

In [ ]:
out["failure_month"].shape

In [ ]:
out["cashflows"][1]

In [ ]:
# diagnostics: run where tickers and prices_m are defined
print("shape:", prices_m.shape)
print("first/last index:", prices_m.index.min(), prices_m.index.max())
print("columns returned:", list(prices_m.columns))
print("missing tickers:", set(tickers) - set(prices_m.columns))
print("NaN counts per column:\n", prices_m.isna().sum())
# show months that are all-NaN (these were dropped by fetcher)
# run before calling fetcher to see raw behavior, otherwise inspect cache file

In [ ]:
prices_m.index.min()

In [ ]:
# align the raw TB3MS series to the rf_m index
tb3 = (
    fetch_fred_series("TB3MS", start=rets_m.index.min(), end=rets_m.index.max())
    .reindex(rf_m.index)
    .iloc[:, 0]
)

# recompute the monthly rate from the raw annualised TB3 quote
rf_from_tb3 = (1.0 + tb3 / 100.0) ** (1.0 / 12.0) - 1.0

rf_a = risk_free_annual(rf_m)

df_rf = pd.DataFrame(
    {
        "tb3_annual_pct": tb3,          # FRED value, annualised percent
        "rf_m": rf_m,                   # monthly decimal rate returned by prep_returns_and_macro
        "rf_from_tb3": rf_from_tb3,     # sanity-check conversion
        "rf_a": np.full(len(rf_m), rf_a),
    }
).rename_axis("date")

df_rf.to_csv("figures/risk_free_check.csv")

print("rf_m matches conversion:", np.allclose(df_rf["rf_m"], df_rf["rf_from_tb3"], rtol=0, atol=1e-12))
print("Unique rf_a value:", df_rf["rf_a"].unique())